In [ ]:
#default_exp density_representers

In [ ]:
#exporti
import math
import torch

In [ ]:
#hide
from nbdev.showdoc import show_doc

# Density representer

In [ ]:
#export
class DensityRepresenter(torch.nn.Module):
    """
    A parent class that inherits different density representers. A density representer has a latent density distribution that is for instance used in the optimization steps of SIMP.
    Additionally, it can be employed for neural reparamterization. Additionally, the density representer has an in-built binarizer, that can be interpreted as a smoothed Heaviside function.
    The binarizer returns densities that have values closer to 0 and 1.
    """
    def __init__(self, 
                 problem:"dl4to.problem.Problem"=None, # The problem object for which the density representer is used. The problem object is necessary to grant that boundary and design space constraints are fulfilled. However, the problem does not need to be passed during initializiaton but can also be passed later by overriding `density_representer.problem`.
                 binarizer_strength:float=1. # The steepness of the smoothed Heaviside-function. A binarizer strength of infinity would corresponds to a non-smooth classical Heaviside step function.
                ):
        super().__init__()
        self.binarizer_strength = binarizer_strength
        self.binarizer_strength_init = binarizer_strength
        self.problem = problem


    def _setup_for_problem(self):
        pass


    @property
    def problem(self):
        return self._problem


    @problem.setter
    def problem(self, problem):
        self._problem = problem
        if self.problem is not None:
            self._setup_for_problem()
            self.reset_binarizer()


    def _apply_density_representer(self):
        raise NotImplementedError("Must be overridden.")


    def _apply_binarizer(self, θ):
        η = .5
        β = self.binarizer_strength
        numerator = math.tanh(β * η) + torch.tanh(β * (θ - η))
        divisor   = math.tanh(β * η) +  math.tanh(β * (1 - η))
        return numerator / divisor


    def steepen_binarizer(self, 
                          binarizer_steepening_factor:float=1.1 # The factor by which to change the current binarizer strength. A value of 1. means that the binarizer does not change.
                         ):
        """
        Increases the binarizer strength by a factor.
        """
        self.binarizer_strength *= binarizer_steepening_factor


    def reset_binarizer(self):
        """
        Resets the binarizer strength to its initial value.
        """
        self.binarizer_strength = self.binarizer_strength_init


    def __call__(self):
        """
        Applies the density representer to the latent density representation, followed by the binarizer step and a projection step. 
        The projection step makes sure that the problem conditions are fulfilled and that the density has no values outside of the unit interval.
        Returns a flattened `torch.Tensor` containing the density distribution.
        """
        θ = self._apply_density_representer()
        if not (torch.all(0 <= θ) and torch.all(θ <= 1)):
            raise ValueError("The function DensityRepresenter_density_representer is only allowed to produce values in [0, 1].")

        θ = self._apply_binarizer(θ)
        θ[self.problem.Ω_design == 0] = 0.
        θ[self.problem.Ω_design == 1] = 1.

        return θ.clamp(0, 1)

In [ ]:
show_doc(DensityRepresenter.steepen_binarizer)

<h4 id="DensityRepresenter.steepen_binarizer" class="doc_header"><code>DensityRepresenter.steepen_binarizer</code><a href="__main__.py#L47" class="source_link" style="float:right">[source]</a></h4>

> <code>DensityRepresenter.steepen_binarizer</code>(**`binarizer_steepening_factor`**:`float`=*`1.1`*)

Increases the binarizer strength by a factor.

||Type|Default|Details|
|---|---|---|---|
|**`binarizer_steepening_factor`**|`float`|`1.1`|The factor by which to change the current binarizer strength. A value of 1. means that the binarizer does not change.|


In [ ]:
show_doc(DensityRepresenter.reset_binarizer)

<h4 id="DensityRepresenter.reset_binarizer" class="doc_header"><code>DensityRepresenter.reset_binarizer</code><a href="__main__.py#L56" class="source_link" style="float:right">[source]</a></h4>

> <code>DensityRepresenter.reset_binarizer</code>()

Resets the binarizer strength to its initial value.

In [ ]:
show_doc(DensityRepresenter.__call__)

<h4 id="DensityRepresenter.__call__" class="doc_header"><code>DensityRepresenter.__call__</code><a href="__main__.py#L63" class="source_link" style="float:right">[source]</a></h4>

> <code>DensityRepresenter.__call__</code>()

Applies the density representer to the latent density representation, followed by the binarizer step and a projection step. 
The projection step makes sure that the problem conditions are fulfilled and that the density has no values outside of the unit interval.
Returns a flattened `torch.Tensor` containing the density distribution.

In [ ]:
#hide
import matplotlib.pyplot as plt
from dl4to.datasets import BasicDataset

In [ ]:
%%time
#hide

def test_that_we_can_apply_binarizer(verbose=True):
    problem = BasicDataset().ledge()
    representer = DensityRepresenter(problem)

    for strength in torch.linspace(1, 19, 7):
        representer.binarizer_strength = strength

        θ = torch.linspace(0, 1, steps=50)
        binarized_θ = representer._apply_binarizer(θ)

        assert torch.all(0 <= binarized_θ)
        assert torch.all(binarized_θ <= 1)

        assert binarized_θ[0] == 0
        assert binarized_θ[-1] == 1

        assert torch.all(binarized_θ[θ < .5] <= θ[θ < .5])
        assert torch.all(binarized_θ[θ > .5] >= θ[θ > .5])

        if verbose:
            plt.plot(θ, binarized_θ, label=strength.item())

    if verbose:
        plt.legend()
        plt.show()


test_that_we_can_apply_binarizer(verbose=False)

CPU times: user 8.15 ms, sys: 3.5 ms, total: 11.6 ms
Wall time: 34.5 ms
